# 목표

식자재 납품 업체에서 소매점으로 납품한 데이터를 이용하여

- 거래처별 일일 객단가추이
- 거래처별 일일 주문품목수 변화추이
- 상품별 일일 매출수량 추이
- 상품 가격별 매출변화 추이
- 담당자별 매출 추이  

을 확인한다.

이후 MLlib을 이용하여 위에서 다룬 내용들에 대한 향후 예측 결과 모델을 만들어본다.

# 아키텍쳐 정의

# 계획

1. 환경구축
    - pyspark 환경 구축
    - mysql 연결
2. 데이터 전처리
    - 데이터 수정
    - 데이터 스키마 수정
    - 불필요한 컬럼 삭제
    - 분석용 데이터 mysql에 적재
3. 개발
    - sql쿼리와 pandas를 이용한 분석 및 시각화
4. 테스트 계획 수립

# Spark 세션 생성

In [1]:
from pyspark.sql import SparkSession
# SparkSession 생성
spark = SparkSession.builder.appName("MiniProject").config("spark.jars", "/usr/local/lib/mysql-connector-java-5.1.49-bin.jar").getOrCreate()
# builder가 세션을 생성하는 객체 : singleton pattern object builder : 여러개 생성되는걸 막아준다

24/12/18 16:15:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


# mysql 연결

In [2]:
host_ip="15.152.104.196"
user_id="spark_user"
user_password="1234"
# mysql_url = f'jdbc:mysql://{host_ip}:3306/my_db'
mysql_url = f'jdbc:mysql://{host_ip}:3306/my_db?useUnicode=true&characterEncoding=UTF-8&characterSetResults=UTF-8'


In [3]:
import os
os.environ["PYSPARK_SUBMIT_ARGS"] = "--jars /usr/local/lib/mysql-connector-java-5.1.49-bin.jar pyspark-shell"

# Data Load

In [4]:
sales_df = spark.read.format("csv")\
    .option("header", 'true')\
    .option('inferSchema', 'true')\
    .load('data/sales_final.csv')

In [6]:
sales_df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Shop_Code: string (nullable = true)
 |-- Shop: string (nullable = true)
 |-- Item_Code: string (nullable = true)
 |-- Item: string (nullable = true)
 |-- Standard: string (nullable = true)
 |-- Unit: string (nullable = true)
 |-- Count: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Sum_Price: string (nullable = true)
 |-- Person: string (nullable = true)



In [7]:
sales_df.show(5)

+----------+---------+-----------+---------+---------------------------+-----------+----+-----+------+---------+------------+
|      Date|Shop_Code|       Shop|Item_Code|                       Item|   Standard|Unit|Count| Price|Sum_Price|      Person|
+----------+---------+-----------+---------+---------------------------+-----------+----+-----+------+---------+------------+
|2024-11-21|   057586|153구포국수|   005466|1.8L 유경빈초밥소스(초대리)|  1.8L(b/6)|  병|    1| 6,760|    6,760| 정동환/세현|
|2024-11-21|   057586|153구포국수|   002803|    1kg 고사리(진공/중국산)|  1kg(b/10)|  봉|    2| 2,250|    4,500| 정동환/세현|
|2024-11-21|   057586|153구포국수|   003305|       ★중국산/슬라이스김치|       10kg|박스|    4|15,490|   61,960| 정동환/세현|
|2024-11-21|   057586|153구포국수|   002685|                *쌀떡볶이中|1.5kg(b/10)|  봉|    5| 2,900|   14,500| 정동환/세현|
|2024-11-21|   057586|153구포국수|   003196|          2kg 쫄면사리/백미|  2kg(b/10)|  줄|    5| 4,050|   20,250| 정동환/세현|
+----------+---------+-----------+---------+---------------------------+---------

# 데이터 전처리

In [ ]:
sales_df = sales_df.drop('Group')

In [8]:
from pyspark.sql.functions import regexp_replace, col

sales_df = sales_df.withColumn("Price", 
    regexp_replace(col("Price"), ",", "").cast("integer"))
sales_df = sales_df.withColumn("Sum_Price", 
    regexp_replace(col("Sum_Price"), ",", "").cast("integer"))

In [9]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, DateType

columns_to_cast = ["Shop_Code", "Item_code", "Count","Price", "Sum_Price"]
for column in columns_to_cast:
    sales_df = sales_df.withColumn(column, col(column).cast(IntegerType()))


In [10]:
sales_df = sales_df.withColumn("Date", col("Date").cast(DateType()))

In [11]:
sales_df.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Shop_Code: integer (nullable = true)
 |-- Shop: string (nullable = true)
 |-- Item_code: integer (nullable = true)
 |-- Item: string (nullable = true)
 |-- Standard: string (nullable = true)
 |-- Unit: string (nullable = true)
 |-- Count: integer (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Sum_Price: integer (nullable = true)
 |-- Person: string (nullable = true)



In [12]:
sales_df.show(5)

+----------+---------+-----------+---------+---------------------------+-----------+----+-----+-----+---------+------------+
|      Date|Shop_Code|       Shop|Item_code|                       Item|   Standard|Unit|Count|Price|Sum_Price|      Person|
+----------+---------+-----------+---------+---------------------------+-----------+----+-----+-----+---------+------------+
|2024-11-21|    57586|153구포국수|     5466|1.8L 유경빈초밥소스(초대리)|  1.8L(b/6)|  병|    1| 6760|     6760| 정동환/세현|
|2024-11-21|    57586|153구포국수|     2803|    1kg 고사리(진공/중국산)|  1kg(b/10)|  봉|    2| 2250|     4500| 정동환/세현|
|2024-11-21|    57586|153구포국수|     3305|       ★중국산/슬라이스김치|       10kg|박스|    4|15490|    61960| 정동환/세현|
|2024-11-21|    57586|153구포국수|     2685|                *쌀떡볶이中|1.5kg(b/10)|  봉|    5| 2900|    14500| 정동환/세현|
|2024-11-21|    57586|153구포국수|     3196|          2kg 쫄면사리/백미|  2kg(b/10)|  줄|    5| 4050|    20250| 정동환/세현|
+----------+---------+-----------+---------+---------------------------+-----------+----+

# 분석용 데이터 MySQL에 적재

In [13]:
sales_df.write \
    .format("jdbc") \
    .option("url", mysql_url) \
    .option("driver", "com.mysql.jdbc.Driver") \
    .option("dbtable", "sales") \
    .option("user", user_id) \
    .option("password", user_password) \
    .option("charset", "UTF-8") \
    .option("useUnicode", "true") \
    .option("characterEncoding", "UTF-8") \
    .mode("overwrite") \
    .save()


Wed Dec 18 16:16:32 KST 2024 WARN: Establishing SSL connection without server's identity verification is not recommended. According to MySQL 5.5.45+, 5.6.26+ and 5.7.6+ requirements SSL connection must be established by default if explicit option isn't set. For compliance with existing applications not using SSL the verifyServerCertificate property is set to 'false'. You need either to explicitly disable SSL by setting useSSL=false, or set useSSL=true and provide truststore for server certificate verification.
Wed Dec 18 16:16:33 KST 2024 WARN: Establishing SSL connection without server's identity verification is not recommended. According to MySQL 5.5.45+, 5.6.26+ and 5.7.6+ requirements SSL connection must be established by default if explicit option isn't set. For compliance with existing applications not using SSL the verifyServerCertificate property is set to 'false'. You need either to explicitly disable SSL by setting useSSL=false, or set useSSL=true and provide truststore for s

## 적재된 데이터 불러와서 확인해보기

In [14]:
new_sales_df = spark.read.format('jdbc').options(url=mysql_url, driver='com.mysql.jdbc.Driver', dbtable='sales', user=user_id, password=user_password).load()

Wed Dec 18 16:16:37 KST 2024 WARN: Establishing SSL connection without server's identity verification is not recommended. According to MySQL 5.5.45+, 5.6.26+ and 5.7.6+ requirements SSL connection must be established by default if explicit option isn't set. For compliance with existing applications not using SSL the verifyServerCertificate property is set to 'false'. You need either to explicitly disable SSL by setting useSSL=false, or set useSSL=true and provide truststore for server certificate verification.


In [15]:
new_sales_df.show(5)

+----------+---------+-----------+---------+---------------------------+-----------+----+-----+-----+---------+------------+
|      Date|Shop_Code|       Shop|Item_code|                       Item|   Standard|Unit|Count|Price|Sum_Price|      Person|
+----------+---------+-----------+---------+---------------------------+-----------+----+-----+-----+---------+------------+
|2024-11-21|    57586|153구포국수|     5466|1.8L 유경빈초밥소스(초대리)|  1.8L(b/6)|  병|    1| 6760|     6760| 정동환/세현|
|2024-11-21|    57586|153구포국수|     2803|    1kg 고사리(진공/중국산)|  1kg(b/10)|  봉|    2| 2250|     4500| 정동환/세현|
|2024-11-21|    57586|153구포국수|     3305|       ★중국산/슬라이스김치|       10kg|박스|    4|15490|    61960| 정동환/세현|
|2024-11-21|    57586|153구포국수|     2685|                *쌀떡볶이中|1.5kg(b/10)|  봉|    5| 2900|    14500| 정동환/세현|
|2024-11-21|    57586|153구포국수|     3196|          2kg 쫄면사리/백미|  2kg(b/10)|  줄|    5| 4050|    20250| 정동환/세현|
+----------+---------+-----------+---------+---------------------------+-----------+----+

Wed Dec 18 16:16:39 KST 2024 WARN: Establishing SSL connection without server's identity verification is not recommended. According to MySQL 5.5.45+, 5.6.26+ and 5.7.6+ requirements SSL connection must be established by default if explicit option isn't set. For compliance with existing applications not using SSL the verifyServerCertificate property is set to 'false'. You need either to explicitly disable SSL by setting useSSL=false, or set useSSL=true and provide truststore for server certificate verification.


In [19]:
new_sales_df.createOrReplaceTempView('sales')

# 분석

## 거래처별 일일 객단가

In [21]:
query = '''
SELECT Date, Shop_Code, Shop, SUM(Sum_Price) AS Daily_Total
FROM sales
WHERE Date = '2024-11-21'
GROUP BY Date, Shop_Code, Shop
order by Daily_Total DESC;
'''
spark.sql(query).show(10)

Wed Dec 18 17:18:00 KST 2024 WARN: Establishing SSL connection without server's identity verification is not recommended. According to MySQL 5.5.45+, 5.6.26+ and 5.7.6+ requirements SSL connection must be established by default if explicit option isn't set. For compliance with existing applications not using SSL the verifyServerCertificate property is set to 'false'. You need either to explicitly disable SSL by setting useSSL=false, or set useSSL=true and provide truststore for server certificate verification.


+----------+---------+---------------------------+-----------+
|      Date|Shop_Code|                       Shop|Daily_Total|
+----------+---------+---------------------------+-----------+
|2024-11-21|    93411|                넥슨 꽃밥집|     586800|
|2024-11-21|    93820|                 족발더하기|     420090|
|2024-11-21|    93539|             정가네숯불갈비|     413170|
|2024-11-21|    96303|병아리김밥 가산디지털단지점|     315072|
|2024-11-21|    92378|        짜글이미식회 화곡점|     279300|
|2024-11-21|    93334|                     홍오로|     261740|
|2024-11-21|    94084|                   국수나무|     257590|
|2024-11-21|    92456|                     우식당|     246060|
|2024-11-21|    94983|             디테크한식뷔페|     244430|
|2024-11-21|    92650|                   카츠애옹|     239960|
+----------+---------+---------------------------+-----------+
only showing top 10 rows



In [22]:
query = '''
SELECT Date, Shop_Code, Shop, SUM(Sum_Price) AS Daily_Total
FROM sales
WHERE Date = '2024-11-22'
GROUP BY Date, Shop_Code, Shop
order by Daily_Total DESC;
'''
spark.sql(query).show(10)

Wed Dec 18 17:18:12 KST 2024 WARN: Establishing SSL connection without server's identity verification is not recommended. According to MySQL 5.5.45+, 5.6.26+ and 5.7.6+ requirements SSL connection must be established by default if explicit option isn't set. For compliance with existing applications not using SSL the verifyServerCertificate property is set to 'false'. You need either to explicitly disable SSL by setting useSSL=false, or set useSSL=true and provide truststore for server certificate verification.


+----------+---------+--------------------------+-----------+
|      Date|Shop_Code|                      Shop|Daily_Total|
+----------+---------+--------------------------+-----------+
|2024-11-22|    91697|                배할머니네|     554360|
|2024-11-22|    93949|                우영푸드빌|     455890|
|2024-11-22|    95153|           오토김밥 대치점|     455200|
|2024-11-22|    95763|       두촌가마솥밥&쭈꾸미|     394980|
|2024-11-22|    92472|                 2.5닭갈비|     382840|
|2024-11-22|    93569|                옥동소뼈탕|     371990|
|2024-11-22|    93816|                  박가부대|     344740|
|2024-11-22|    92374|         옥사장집밥 화곡점|     330790|
|2024-11-22|    94759|                  광동포차|     306900|
|2024-11-22|    94309|밥한민국 구구족건물지하1층|     301290|
+----------+---------+--------------------------+-----------+
only showing top 10 rows



## 거래처별 일일 구매 수량

In [23]:
query = '''
SELECT Date, Shop_Code, Shop, SUM(Count) AS Daily_Count
FROM sales
WHERE Date = '2024-11-21'
GROUP BY Date, Shop_Code, Shop
order by Daily_Count DESC;
'''
spark.sql(query).show(10)

Wed Dec 18 17:19:03 KST 2024 WARN: Establishing SSL connection without server's identity verification is not recommended. According to MySQL 5.5.45+, 5.6.26+ and 5.7.6+ requirements SSL connection must be established by default if explicit option isn't set. For compliance with existing applications not using SSL the verifyServerCertificate property is set to 'false'. You need either to explicitly disable SSL by setting useSSL=false, or set useSSL=true and provide truststore for server certificate verification.


+----------+---------+-----------------------------------+-----------+
|      Date|Shop_Code|                               Shop|Daily_Count|
+----------+---------+-----------------------------------+-----------+
|2024-11-21|    93411|                        넥슨 꽃밥집|         72|
|2024-11-21|    92456|                             우식당|         70|
|2024-11-21|    94087|최고당돈까스 안양아이에스비즈타워점|         67|
|2024-11-21|    92722|                           인생면가|         53|
|2024-11-21|    93820|                         족발더하기|         51|
|2024-11-21|    92378|                짜글이미식회 화곡점|         50|
|2024-11-21|    92814|                    담소 소사골순대|         50|
|2024-11-21|    93479|                      우화 숯불구이|         33|
|2024-11-21|    91941|                           국밥상회|         33|
|2024-11-21|    57586|                        153구포국수|         32|
+----------+---------+-----------------------------------+-----------+
only showing top 10 rows



In [24]:
query = '''
SELECT Date, Shop_Code, Shop, SUM(Count) AS Daily_Count
FROM sales
WHERE Date = '2024-11-22'
GROUP BY Date, Shop_Code, Shop
order by Daily_Count DESC;
'''
spark.sql(query).show(10)

Wed Dec 18 17:19:17 KST 2024 WARN: Establishing SSL connection without server's identity verification is not recommended. According to MySQL 5.5.45+, 5.6.26+ and 5.7.6+ requirements SSL connection must be established by default if explicit option isn't set. For compliance with existing applications not using SSL the verifyServerCertificate property is set to 'false'. You need either to explicitly disable SSL by setting useSSL=false, or set useSSL=true and provide truststore for server certificate verification.


+----------+---------+-------------------------------+-----------+
|      Date|Shop_Code|                           Shop|Daily_Count|
+----------+---------+-------------------------------+-----------+
|2024-11-22|    96216|          곱창전골 1층 간판없음|        164|
|2024-11-22|    92472|                      2.5닭갈비|         87|
|2024-11-22|    94579|                       인생푸드|         73|
|2024-11-22|    92529|206숯불구이.     이공육숯불구이|         71|
|2024-11-22|    94309|     밥한민국 구구족건물지하1층|         68|
|2024-11-22|    95408|                         아성원|         67|
|2024-11-22|    91697|                     배할머니네|         65|
|2024-11-22|    93816|                       박가부대|         58|
|2024-11-22|    92880|                       꽃삼살롱|         46|
|2024-11-22|    92515|              최우영수산 영등포|         45|
+----------+---------+-------------------------------+-----------+
only showing top 10 rows



## 거래처별 일일 객단가추이

In [25]:
query = '''
SELECT 
    ss.Shop_Code,
    ss.Shop,
    ss.First_Purchase,
    ss.Last_Purchase,
    ss.Purchase_Days,
    ss.Date_Range,
    ss.Avg_Days_Between_Purchases,
    dt.Date,
    COALESCE(dt.Daily_Total, 0) AS Daily_Total
FROM 
    (SELECT 
        Shop_Code,
        Shop,
        MIN(Date) AS First_Purchase,
        MAX(Date) AS Last_Purchase,
        COUNT(DISTINCT Date) AS Purchase_Days,
        DATEDIFF(MAX(Date), MIN(Date)) AS Date_Range,
        DATEDIFF(MAX(Date), MIN(Date)) / (COUNT(DISTINCT Date) - 1) AS Avg_Days_Between_Purchases
    FROM 
        sales
    GROUP BY 
        Shop_Code, Shop
    HAVING 
        COUNT(DISTINCT Date) > 1) ss
LEFT JOIN 
    (SELECT 
        Date, 
        Shop_Code,
        SUM(Sum_Price) AS Daily_Total
    FROM 
        sales
    GROUP BY 
        Date, Shop_Code) dt 
ON ss.Shop_Code = dt.Shop_Code
ORDER BY 
    ss.Shop_Code, dt.Date;
'''
spark.sql(query).show(10)

Wed Dec 18 17:20:27 KST 2024 WARN: Establishing SSL connection without server's identity verification is not recommended. According to MySQL 5.5.45+, 5.6.26+ and 5.7.6+ requirements SSL connection must be established by default if explicit option isn't set. For compliance with existing applications not using SSL the verifyServerCertificate property is set to 'false'. You need either to explicitly disable SSL by setting useSSL=false, or set useSSL=true and provide truststore for server certificate verification.
Wed Dec 18 17:20:27 KST 2024 WARN: Establishing SSL connection without server's identity verification is not recommended. According to MySQL 5.5.45+, 5.6.26+ and 5.7.6+ requirements SSL connection must be established by default if explicit option isn't set. For compliance with existing applications not using SSL the verifyServerCertificate property is set to 'false'. You need either to explicitly disable SSL by setting useSSL=false, or set useSSL=true and provide truststore for s

+---------+------------------------+--------------+-------------+-------------+----------+--------------------------+----------+-----------+
|Shop_Code|                    Shop|First_Purchase|Last_Purchase|Purchase_Days|Date_Range|Avg_Days_Between_Purchases|      Date|Daily_Total|
+---------+------------------------+--------------+-------------+-------------+----------+--------------------------+----------+-----------+
|    25907|두남자의 볶음밥 & 냉모밀|    2024-11-21|   2024-11-22|            2|         1|                       1.0|2024-11-21|      85660|
|    25907|두남자의 볶음밥 & 냉모밀|    2024-11-21|   2024-11-22|            2|         1|                       1.0|2024-11-22|     162570|
|    29256|                광장닭발|    2024-11-21|   2024-11-22|            2|         1|                       1.0|2024-11-21|     155700|
|    29256|                광장닭발|    2024-11-21|   2024-11-22|            2|         1|                       1.0|2024-11-22|      70470|
|    91004|                    솔솥|    202

## 거래처별 일일 주문품목수 변화추이

In [30]:
print('*******************************************************')
print('Daily_Distinct_Items: 해당 날짜에 거래처에서 주문한 고유한 품목의 수')
print('Daily_Count: 해당 날짜의 거래처의 총 주문 수량')
print('*******************************************************')
query = '''
SELECT 
    Date,
    Shop_Code,
    Shop,
    COUNT(DISTINCT Item_code) AS Daily_Distinct_Items,
    SUM(Count) AS Daily_Count
FROM 
    sales
GROUP BY 
    Date, Shop_Code, Shop
ORDER BY 
    Shop_Code, Date;
'''
spark.sql(query).show(20)

*******************************************************
Daily_Distinct_Items: 해당 날짜에 거래처에서 주문한 고유한 품목의 수
Daily_Count: 해당 날짜의 거래처의 총 주문 수량
*******************************************************


Wed Dec 18 17:43:16 KST 2024 WARN: Establishing SSL connection without server's identity verification is not recommended. According to MySQL 5.5.45+, 5.6.26+ and 5.7.6+ requirements SSL connection must be established by default if explicit option isn't set. For compliance with existing applications not using SSL the verifyServerCertificate property is set to 'false'. You need either to explicitly disable SSL by setting useSSL=false, or set useSSL=true and provide truststore for server certificate verification.


+----------+---------+------------------------+--------------------+-----------+
|      Date|Shop_Code|                    Shop|Daily_Distinct_Items|Daily_Count|
+----------+---------+------------------------+--------------------+-----------+
|      null|     null|                        |                   0|       null|
|2024-11-22|    17001|             모꼬지 식당|                   5|          9|
|2024-11-21|    25907|두남자의 볶음밥 & 냉모밀|                   6|         13|
|2024-11-22|    25907|두남자의 볶음밥 & 냉모밀|                  12|         29|
|2024-11-21|    26131|                찜꽁찜닭|                   7|         14|
|2024-11-22|    27001|                곱창주막|                   7|         12|
|2024-11-22|    27600|                대령숙수|                   4|         10|
|2024-11-21|    29256|                광장닭발|                   8|         22|
|2024-11-22|    29256|                광장닭발|                   7|         16|
|2024-11-21|    32554|                손두부가|                   7|         

In [ ]:
spark.stop()